# Sentence2sentence generation based on OpenSubtitles
Inspired by the article: https://arxiv.org/abs/1506.05869 (https://arxiv.org/pdf/1506.05869.pdf)

See: http://www.opensubtitles.org/ and http://opus.lingfil.uu.se/OpenSubtitles.php

Model: 
- 2-layered LSTM (4096 memory cells each).
- AdaGrad with gradient clipping. 
- Vocabulary most frequent 100K words.
- To speed-up softmax: project memory cells to 2048 linear units before feeding info to the classifier.

Predict the next sentence (token-by-token) by several ones before it. 

"Greedy" inference approach: feed the predicted output token as input to predict the next output (an alternative could be a "beam search").

## Prepare data

To get data run ./setup.sh from OpenSubtitles parser: https://github.com/inikdom/opensubtitles-parser
It will generate a bunch of raw text files with sentences

In the original article they had: movie conversations in XML, training - 62M sentences (923M tokens), validation - 26M sentences (395M tokens).

In [24]:
import glob
import numpy as np 

raw_files_pattern = "data/opensubtitles/*.txt" 

seq2seq_data = []
for filename in glob.glob(raw_files_pattern):
    with open(filename, "r") as file:
        lines = file.read().splitlines()
        file_seq2seq = list(zip(lines[:-1], lines[1:]))
        seq2seq_data.extend(file_seq2seq)
print(seq2seq_data[:10])
len(seq2seq_data)

[('25 . 000', 'catch a falling star'), ('catch a falling star', 'and put it in your pocket'), ('and put it in your pocket', 'never iet itfade away'), ('never iet itfade away', 'catch afailing star and put it in your pocket ...'), ('catch afailing star and put it in your pocket ...', 'you know how in some movies ... they have a dream sequence , only they dontteii you its a dream ?'), ('you know how in some movies ... they have a dream sequence , only they dontteii you its a dream ?', 'this is so not a dream .'), ('this is so not a dream .', 'itwasnt supposed to be iike this .'), ('itwasnt supposed to be iike this .', 'i was justtrying to do my job , and then things happened .'), ('i was justtrying to do my job , and then things happened .', 'weil , iife happened .'), ('weil , iife happened .', 'and now im here .')]


2663918